PyTorch神经网络建模：
- 数据准备
- 模型建立
- 模型训练
- 模型评估使用和保存

# 数据准备

数据准备是PyTorch网络中非常重要且比较困难的一个部分，实际任务中一般会面临结构化数据、图片数据、文本数据和时间序列数据，不同的数据会有不同的数据准备方法。

本文总结针对这四种数据进行准备，每种数据又有哪些不同的建模方法。

# Dataset与DataLoader

PyTorch通常使用Dataset和DataLoader这两个工具类构建数据管道，建模各种数据避不开这两个工具类。

- Dataset定义数据集的内容，类似于列表的数据结构，具有确定的长度，能够用索引获取数据集中的元素。
  - 绝大多数情况下，只需实现Dataset的\_\_len\_\_方法和\_\_getitem\_\_方法，就可以轻松构建自己的数据集，并用默认数据管道进行加载
- DataLoader定义按batch加载数据集的方法，实现一个\_\_iter\_\_方法的可迭代对象，每次迭代输出一个batch数据。
  - DataLoader可以控制batch大小，batch中元素的采样方法，以及将batch结果整理成模型所需输入形式的方法，同时可以多进程读取数据。

## 概述

通过这两个工具类完成数据管道的构建，有两个问题：
- 如何获取一个batch？
- Dataset与DataLoader是如何分工合作的？

### 如何获取一个batch

数据集特征X，标签Y，则数据集可以表示成（X，Y），batch大小为m，则获取一个batch步骤：
- 确定数据集的长度n
  - 一共有多少个样本，这样指定每个batch时，计算机可以规划分为几个batch
- 从[0, n-1]范围内抽样m个数（batch大小）
  - 假设m=4，则拿到的结果就是一个列表，类似：indics=[1, 4, 8, 9]
- 根据下表从数据集中取m个数对应的下标元素
  - 拿到一个元组列表，类似：samples=[(X[1], Y[1]), (X[4], Y[4]), (X[8], Y[8]), (X[9], Y[9])]
- 将结果整理成两个张量作为输出：
  - 结果时两个张量，类似：batch=(features, labels)，其中：
  - features=torch.stack(X[1], X[4], X[8], X[9])
  - labels=torch.stack(Y[1], Y[4], Y[8], Y[9])

这样，就完成了一个batch数据获取操作。

### Dataset与DataLoader功能分工

<img style="float: center;" src="images/1.png" width="70%">

上述流程图，把DataLoader读取数据的流程梳理一遍。

DataLoader的作用就是构建一个数据装载器，根据提供的batch_size大小，将数据样本分成一个个batch去训练模型，这个分的过程中需要把数据取到（需要借助Dataset的getitem）：
- 第一个步骤：数据的总长度，这个需要在Dataset的\_\_len\_\_方法中告诉计算机
- 第二个步骤：0到n-1范围中抽样出m个数的方法，由DataLoader的sampler和batch_sampler参数指定：
  - sampler参数指定单个元素抽样方法（一般无需设置），程序默认在DataLoader的参数shuffle=True时采用随机抽样，shuffle=False时采用顺序抽样。
  - batch_sampler参数将多个抽样的元素整理成一个列表（一般无需设置），默认方法在DataLoader的参数drop_last=True时会丢弃数据集最后一个长度不能被batch大小整除的批次，在drop_last=False时保留最后一个批次。
- 第三个步骤：根据下标取数据集中的元素（需要自己写读取函数），由Dataset的\_\_getitem\_\_方法实现，这个函数接收的参数是一个索引。
- 第四个步骤：逻辑由DataLoader的参数collate_fn指定，一般无需设置。

## 使用

### Dataset创建数据集

Dataset核心接口逻辑伪代码

In [1]:
class Dataset(object):
    def __init__(self):
        pass
    
    def __len__(self):
        raise NotImplementedError
        
    def __getitem__(self,index):
        raise NotImplementedError

Dataset创建数据集常用方法：
- torch.utils.data.TensorDataset：根据Tensor创建数据集（Numpy的array和Pandas的DataFrame需要先转换成Tensor）
- torchvision.datasets.ImageFolder：根据图片目录创建图片数据集
- torch.utils.data.Dataset：创建自定义数据集（需要实现len和getitem方法）
- torch.utils.data.random_split：将一个数据集分割成多份（训练集，验证集，测试机）
- 调用Dataset加法运算符（+）将多个数据集合并成一个数据集

### DataLoader加载数据集

DataLoader逻辑接口代码

In [2]:
class DataLoader(object):
    def __init__(self,dataset,batch_size,collate_fn,shuffle = True,drop_last = False):
        self.dataset = dataset
        self.sampler =torch.utils.data.RandomSampler if shuffle else \
           torch.utils.data.SequentialSampler
        self.batch_sampler = torch.utils.data.BatchSampler
        self.sample_iter = self.batch_sampler(
            self.sampler(range(len(dataset))),
            batch_size = batch_size,drop_last = drop_last)
        
    def __next__(self):
        indices = next(self.sample_iter)
        batch = self.collate_fn([self.dataset[i] for i in indices])
        return batch

DataLoader能够控制batch的大小，batch中元素的采样方法，以及将batch结果整理成模型所输入形式的方法，并且能够使用多进程读取数据。

DataLoader的函数签名如下：
```python
DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    sampler=None,
    batch_sampler=None,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
    drop_last=False,
    timeout=0,
    worker_init_fn=None,
    multiprocessing_context=None,
)
```

通常仅配置dataset, batch_size, shuffle, num_workers, drop_last五个参数，其他参数使用默认值即可。

DataLoader除了可以加载torch.utils.data.Dataset外，还可以加载另外一种数据集：
- torch.utils.data.IterableDataset
- Dataset数据集相当于一种列表结构，IterableDataset相当于一种迭代器结构，其更加复杂，一般较少使用。
- 参数说明：
  - dataset：数据集
  - batch_size：批次大小
  - shuffle：是否乱序
  - sampler：样本采样函数，一般无需设置
  - batch_sampler：批次采样函数，一般无需设置
  - num_workers：使用多进程读取数据，设置的进程数
  - collate_fn：整理一个批次数据的函数。
  - pin_memory：是否设置为锁业内存。默认False，锁业内存不会使用虚拟内存(硬盘)，从锁业内存拷贝到GPU上速度会更快。
  - drop_last：是否丢弃最后一个样本数量不足batch_size批次数据。
  - timeout：加载一个数据批次的最长等待时间，一般无需设置。
  - worker_init_fn：每个worker中dataset的初始化函数，常用于IterableDataset。

In [3]:
import torch.utils.data as tud
import torch

#构建输入数据管道
ds = tud.TensorDataset(torch.arange(1,50))
dl = tud.DataLoader(ds,
                batch_size = 10,
                shuffle= True,
                num_workers=2,
                drop_last = True)
#迭代数据
for batch, in dl:
    print(batch)

tensor([49, 14, 33,  8, 22, 29, 28, 20, 27, 19])
tensor([ 7, 26, 25, 48,  6, 36,  5, 12, 45, 46])
tensor([ 9, 13, 17,  1, 24, 39, 11, 15, 16, 32])
tensor([47, 44, 21, 31, 23, 34, 35, 43,  4, 38])


# 结构化数据建模

将结构化数据封装成DataLoader形式，形成可迭代的数据管道，模型训练时，可以遍历DataLoader对象去取数据。

结构化数据有三种建模方式

## 直接用TensorDataset和DataLoader封装成可迭代的数据管道

只要把结构化的数据预处理完毕，做成训练集和测试集之后，直接拿这两个函数进行封装

In [4]:
import pandas as pd

# 首先， 读入数据
dftrain_raw = pd.read_csv('data/titanic/train.csv')
dftest_raw = pd.read_csv('data/titanic/test.csv')

可以看到数据的样子，典型的结构化数据：
<img style="float: center;" src="images/2.png" width="70%">

进行简单的数据预处理，划分出训练集和测试集

In [5]:
def preprocessing(dfdata):
    dfresult= pd.DataFrame()
    #Pclass
    dfPclass = pd.get_dummies(dfdata['Pclass'])
    dfPclass.columns = ['Pclass_' +str(x) for x in dfPclass.columns ]
    dfresult = pd.concat([dfresult,dfPclass],axis = 1)

    #Embarked
    dfEmbarked = pd.get_dummies(dfdata['Embarked'],dummy_na=True)
    dfEmbarked.columns = ['Embarked_' + str(x) for x in dfEmbarked.columns]
    dfresult = pd.concat([dfresult,dfEmbarked],axis = 1)

    return(dfresult)

x_train = preprocessing(dftrain_raw).values   # x_train.shape = (712, 15)
y_train = dftrain_raw.Survived.values    # y_train.shape = (712, 1)

x_test = preprocessing(dftest_raw).values   # x_test.shape = (179, 15)
y_test = dftest_raw.Survived.values     # y_test.shape = (179, 1)

数据处理完毕后，可以发现训练集样本是712,特征数是15，之后就可以构建数据管道了

In [8]:
dl_train = tud.DataLoader(tud.TensorDataset(torch.tensor(x_train).float(),torch.tensor(y_train).float()),
                     shuffle = True, batch_size = 8)
dl_valid = tud.DataLoader(tud.TensorDataset(torch.tensor(x_test).float(),torch.tensor(y_test).float()),
                     shuffle = False, batch_size = 8)

# 我们可以测试一下数据管道
for features,labels in dl_train:
    print(features,labels)
    break

tensor([[1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0.]]) tensor([1., 0., 1., 0., 0., 0., 0., 1.])


## 自己构造数据管道迭代器，不通过Pytorch提供的接口

自己写如何划分数据，首先获取全部数据的个数，然后生成索引，根据提供的batch大小把索引划分开，返回对应的数据

In [12]:
import numpy as np

# 样本数量
n = 400

# 生成测试用数据集
X = 10 * torch.rand([n, 2]) - 5.0  #torch.rand是均匀分布 
w0 = torch.tensor([[2.0], [-3.0]])
b0 = torch.tensor([[10.0]])
Y = X @ w0 + b0 + torch.normal(0.0, 2.0, size = [n, 1])  # @表示矩阵乘法,增加正态扰动

数据集有400个样本，2个特征，构建数据管道

In [13]:
def data_iter(features, labels, batch_size=8):
    num_examples = len(features)
    indices = list(range(num_examples))
    np.random.shuffle(indices)  # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        indexs = torch.LongTensor(indices[i: min(i + batch_size, num_examples)])
        yield  features.index_select(0, indexs), labels.index_select(0, indexs)

模型训练的时候，直接遍历即可

In [14]:
for feature, label in data_iter(X, Y, batch_size=8):
    print(feature)
    print(label)
    break

tensor([[-1.4392, -4.0170],
        [ 2.3910,  2.2930],
        [ 3.6774,  2.5477],
        [ 1.5018,  0.4600],
        [ 2.1547,  0.6044],
        [ 4.5885, -0.9907],
        [ 4.5781,  4.5254],
        [-2.4320,  2.4013]])
tensor([[19.4501],
        [ 9.5968],
        [12.1262],
        [13.4225],
        [11.8993],
        [22.1154],
        [ 7.3442],
        [-0.8488]])


# 时序数据建模

时序数据建模需要自定义数据集，涉及自回归问题，也就是用到了历史的数据作为特征。

这种数据一般会采用滑动窗口把原数据集进行一个切割获得X和Y。

下面这个数据集是中国2020年3月之前的疫情数据，典型的时间序列数据：
<img style="float: center;" src="images/3.png" width="70%">

特征是确诊人数，治愈人数和死亡人数，预测是接下来的确诊人数，治愈人数和死亡人数。（自回归）

通过继承torch.utils.data.Dataset实现自定义事件序列数据集：
- \_\_len\_\_：实现len(dataset)返回整个数据集的大小
- \_\_getitem\_\_：获取一些索引数据，使用dataset[i]返回数据集中的第i个样本

必须要覆盖这两个方法。

In [17]:
# 用某日前8天窗口数据作为输入预测该日数据
WINDOW_SIZE = 8

class Covid19Dataset(tud.Dataset):
    def __len__(self):
        return len(dfdiff) - WINDOW_SIZE
    
    def __getitem__(self,i):
        x = dfdiff.loc[i:i+WINDOW_SIZE-1,:]
        feature = torch.tensor(x.values)
        y = dfdiff.loc[i+WINDOW_SIZE,:]
        label = torch.tensor(y.values)
        return (feature,label)
    
ds_train = Covid19Dataset()

#数据较小，可以将全部训练数据放入到一个batch中，提升性能
dl_train = tud.DataLoader(ds_train, batch_size = 38)

这是比较常规的时序建模方式，下面是一个非线性自回归的一个自定义数据集的方式，更加复杂一些。

非线性自回归建模：既涉及到普通的结构化数据，也涉及到时序数据。
<img style="float: center;" src="images/4.png" width="70%">

要预测未来7天的温度，既涉及到温度本身的特征，也涉及到其他特征来影响温度，问题的建模思路与上面一致，基于过去一段时间的所有特征，但是写法与上面不同，不是直接一个滑动窗口去切，而是先把这些滞后特征做成一行，然后再进行shape转换。

这个比较复杂的就是滞后特征的提取，这个灵活性更高，可以在series_to_supervised函数里做各种处理工作。

In [18]:
class TemperatureDataSet(Dataset):
    def __init__(self, n_in, n_out):
        """
        函数用途： 加载数据并且初始化
        参数说明：
            n_in: 用多长时间进行预测
            n_out: 预测多长时间
        """
        # 读取数据，去除缺失严重属性，插值
        df = pd.read_csv("dsw.csv", index_col=0)
        df.drop(['pH'], axis=1, inplace=True)
        df.index = pd.to_datetime(df.index)
        df.interpolate(method='time', inplace=True)  # 时间插值

        # 标准化
        self.ss = StandardScaler()
        self.std_data = self.ss.fit_transform(df['Temperature'].values.reshape(-1, 1))

        # 转化为监督测试集
        data = self.series_to_supervised(self.std_data, n_in, n_out)
        re_data = self.series_to_supervised(df['Temperature'].values.reshape(-1, 1), n_in, n_out)
        self.x = torch.from_numpy(data.iloc[:, :n_in].values).view(-1, n_in, 1)
        self.y = torch.from_numpy(re_data.iloc[:, n_in:].values).view(-1, n_out, 1)
        self.len = self.x.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, item):
        return self.x[item, :].type(torch.FloatTensor), self.y[item, :].type(torch.FloatTensor)

    def series_to_supervised(self, data, n_in=1, n_out=1, dropnan=True):
        """
        函数用途：将时间序列转化为监督学习数据集。
        参数说明：
            data: 观察值序列，数据类型可以是 list 或者 NumPy array。
            n_in: 作为输入值(X)的滞后组的数量。
            n_out: 作为输出值(y)的观察组的数量。
            dropnan: Boolean 值，确定是否将包含 NaN 的行移除。
        返回值:
            经过转换的用于监督学习的 Pandas DataFrame 序列。
        """
        n_vars = 1 if type(data) is list else data.shape[1]
        df = pd.DataFrame(data)
        cols, names = list(), list()
        # 输入序列 (t-n, ... t-1)
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
        # 预测序列 (t, t+1, ... t+n)
        for i in range(0, n_out):dsw.csv
            cols.append(df.shift(-i))
            if i == 0:
                names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
            else:
                names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
        # 将所有列拼合
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop 掉包含 NaN 的行
        if dropnan:
            agg.dropna(inplace=True)
        return agg


data_set = TemperatureDataSet(n_in=10, n_out=5)

FileNotFoundError: [Errno 2] No such file or directory: 'dsw.csv'

# 图片数据建模

PyTorch中构建图片数据管道通常有两种方法：
1. torchvision中的datasets.ImageFolder读取图片然后用DataLoader来并行加载。
2. 继承torch.utils.data.Dataset实现用户自定义读取逻辑然后用DataLoader并行加载

## torchvision中的datasets.ImageFloder

torchvision是一个计算机视觉工具包，需要在安装PyTorch后单独安装这个包，其有三个模块：
- torchvision.transforms：常用的图像预处理方法，比如标准化、中心化、旋转、翻转等操作
- torchvision.datasets：常用的数据集dataset实现，MNIST，CIFAR-10，ImageNet等
- torchvision.models：常用模型预训练，AlexNet，VGG，ResNet，GoogLeNet

这里以CIFAR-2数据集作为示例，训练集有airplane和automobile图片各5000张，测试集照片各1000张，任务的目标是训练一个模型来对airplane和automobile进行分类。
<img style="float: center;" src="images/5.png" width="70%">

使用ImageFloder读取数据集，先定义数据的转换格式，图片数据涉及各种数据预处理，比如转成张量，裁剪，旋转，变换等操作，先定义这些操作，之后在读取数据集的时候，通过transform参数把这些处理传入，就可以直接处理。

In [ ]:
# 定义预处理方式
# Compose里可以放更多的处理方式
transform_train = transforms.Compose(
    [transforms.ToTensor()])
transform_valid = transforms.Compose(
    [transforms.ToTensor()])

# 读取数据
ds_train = datasets.ImageFolder("./data/cifar2/train/",
            transform = transform_train,target_transform= lambda t:torch.tensor([t]).float())
ds_valid = datasets.ImageFolder("./data/cifar2/test/",
            transform = transform_train,target_transform= lambda t:torch.tensor([t]).float())
 
# 封装成数据管道
dl_train = DataLoader(ds_train,batch_size = 50,shuffle = True,num_workers=3)
dl_valid = DataLoader(ds_valid,batch_size = 50,shuffle = True,num_workers=3)

## 用户自定义读取逻辑用DataLoader加载

这里数据形式与上面一样：
<img style="float: center;" src="images/6.png" width="70%">

类别分开，每一类里都是图片数据，首先定义一个自己写的Dataset类读取数据，生成数据集，实现逻辑依然是\_\_getitem\_\_里可以拿到某个索引对应的数据，注意这个函数接收的参数是一个索引，返回这个索引对应的数据。

In [ ]:
class RMBDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        rmb面额分类任务的Dataset
        :param data_dir: str, 数据集所在路径
        :param transform: torch.transform，数据预处理
        """
        self.label_name = {"1": 0, "100": 1}
        self.data_info = self.get_img_info(data_dir)  # data_info存储所有图片路径和标签，在DataLoader中通过index读取样本
        self.transform = transform

    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        img = Image.open(path_img).convert('RGB')     # 0~255

        if self.transform is not None:
            img = self.transform(img)   # 在这里做transform，转为tensor等等

        return img, label

    def __len__(self):
        return len(self.data_info)

    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                img_names = os.listdir(os.path.join(root, sub_dir))
                img_names = list(filter(lambda x: x.endswith('.jpg'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    label = rmb_label[sub_dir]
                    data_info.append((path_img, int(label)))

        return data_info

之后用自定义的Datasets，构建数据管道

In [ ]:
# transforms模块，进行数据预处理
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

## 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)

# 文本数据建模

文本数据预处理较为繁琐，包括中文切词，构建词典，编码转换，序列填充，构建数据管道等等。

PyTorch中对文本数据建模一般采用两种方式：
- torchtext包：可以构建文本分类，序列标注，问答模型，机器翻译等NLP任务数据集
- 自定义Dataset，处理比较繁琐

IMDB数据集的目标是根据电影评论的文本内容预测评论的情感标签：
- 训练集有2w条电影评论文本
- 测试集有5k条电影评论文本
- 其中正负评论各占一半
<img style="float: center;" src="images/7.png" width="70%">

## torchtext包进行文本分类数据建模

- torchtext.data.Example：表示一个样本，数据和标签
- torchtext.vocab.Vocab：词汇表，可以导入一些预训练词向量
- torchtext.data.Datasets：数据集类，\_\_getitem\_\_返回Example实例。
- torchtext.data.Field：定义字段的处理方法（文本字段，标签字段）创建 Example时的预处理，batch时的一些处理操作。
- torchtext.data.Iterator：迭代器，生成batch
- torchtext.datasets：包含了常见的数据集

操作逻辑，先从Field里定义一些字段预处理方式，然后用Datasets里的子类构建数据集，构建词典，把数据用Iterator封装成数据迭代器。

In [ ]:
MAX_WORDS = 10000
MAX_LEN = 200
BATCH_SIZE = 20 

# string.punctuation表示所有的标点字符， 下面这句话就是把每个句子里面的所有标点符号都替换成空字符， 然后按照空格进行分词
tokenize = lambda x: re.sub('[%s]' % string.punctuation, "", x).split(" ")  # 字符串的替换

def filterLowFreqWords(arr, vocab):
    arr = [[x if x<MAX_WORDS else 0 for x in example] for example in arr]
    return arr

# 定义各个字段的预处理方法
# sequential告诉它输入是序列的形式， lower等于True， 转成小写  postprocessing=False这块不知道啥意思
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=MAX_LEN, postprocessing=filterLowFreqWords)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)   
#field在默认的情况下都期望一个输入是一组单词的序列，并且将单词映射成整数。这个映射被称为vocab。
#如果一个field已经被数字化了并且不需要被序列化，可以将参数设置为use_vocab=False以及sequential=False。


# 构建表格型dataset  
# torchtext.data.TabularDataset可读取csv, tsv, json等格式
ds_train, ds_test = torchtext.data.TabularDataset.splits(
    path='./data/imdb', train='train.tsv', test='test.tsv', format='tsv',
    fields=[('label', LABEL), ('text', TEXT)], skip_header=False
)

# 构建词典
TEXT.build_vocab(ds_train)

# 构建数据管道迭代器
train_iter, test_iter = torchtext.data.Iterator.splits(
    (ds_train, ds_test), sort_within_batch=True, sort_key=lambda x: len(x.text),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE)
)
# 查看数据
for batch in train_iter:
    features = batch.text
    labels = batch.label
    print(features)
    print(features.shape)
    print(labels)
    break

## 自定义文本数据集

思路：先对训练文本分词构建词典，然后将训练集文本和测试集文本数据转换成token单词编码，之后转换成单词编码的训练集数据和测试集数据按样本分割成多个文件，一个文件代表一个样本。最后根据文件名列表获取对应序号的样本内容，从而构建Dataset数据集。

In [ ]:
# 定义一些变量
MAX_WORDS = 10000  # 仅考虑最高频的10000个词
MAX_LEN = 200  # 每个样本保留200个词的长度
BATCH_SIZE = 20 

train_data_path = 'data/imdb/train.tsv'
test_data_path = 'data/imdb/test.tsv'
train_token_path = 'data/imdb/train_token.tsv'
test_token_path =  'data/imdb/test_token.tsv'
train_samples_path = 'data/imdb/train_samples/'
test_samples_path =  'data/imdb/test_samples/'

我们构建词典， 并保留最高频个词：

In [ ]:
##构建词典
word_count_dict = {}    

#清洗文本
def clean_text(text):
    lowercase = text.lower().replace("\n"," ")   # 转成小写
    stripped_html = re.sub('<br />', ' ',lowercase)    # 正则修改
    cleaned_punctuation = re.sub('[%s]'%re.escape(string.punctuation),'',stripped_html)  # 去掉其他符号
    return cleaned_punctuation

with open(train_data_path,"r",encoding = 'utf-8') as f:
    for line in f:
        label,text = line.split("\t")     # label和句子分开
        cleaned_text = clean_text(text)      # 清洗文本
        for word in cleaned_text.split(" "):
            word_count_dict[word] = word_count_dict.get(word,0)+1  # 统计单词频数

df_word_dict = pd.DataFrame(pd.Series(word_count_dict,name = "count"))
df_word_dict = df_word_dict.sort_values(by = "count",ascending =False)

df_word_dict = df_word_dict[0:MAX_WORDS-2] #  
df_word_dict["word_id"] = range(2,MAX_WORDS) #编号0和1分别留给未知词<unkown>和填充<padding>

word_id_dict = df_word_dict["word_id"].to_dict()

利用构建好的词典，将文本转成token序号

In [ ]:
#转换token
# 填充文本
def pad(data_list,pad_length):
    padded_list = data_list.copy()
    if len(data_list)> pad_length:    # 如果句子里面的单词个数比字典长度大， 那就取后面字典长度个大小
         padded_list = data_list[-pad_length:]
    if len(data_list)< pad_length:   # 如果句子小， 前面填充1， 弄到字典长度大小
         padded_list = [1]*(pad_length-len(data_list))+data_list
    return padded_list

def text_to_token(text_file,token_file):
    with open(text_file,"r",encoding = 'utf-8') as fin,\
      open(token_file,"w",encoding = 'utf-8') as fout:
        for line in fin:
            label,text = line.split("\t")
            cleaned_text = clean_text(text)
            word_token_list = [word_id_dict.get(word, 0) for word in cleaned_text.split(" ")]  # 把单词转换成词典中的位置
            pad_list = pad(word_token_list,MAX_LEN)
            out_line = label+"\t"+" ".join([str(x) for x in pad_list])
            fout.write(out_line+"\n")
        
text_to_token(train_data_path,train_token_path)
text_to_token(test_data_path,test_token_path)

接着将token文本按照样本分割，每个文件存放一个样本的数据

In [ ]:
# 分割样本
import os

if not os.path.exists(train_samples_path):
    os.mkdir(train_samples_path)
    
if not os.path.exists(test_samples_path):
    os.mkdir(test_samples_path)
    
def split_samples(token_path,samples_dir):
    with open(token_path,"r",encoding = 'utf-8') as fin:
        i = 0
        for line in fin:
            with open(samples_dir+"%d.txt"%i,"w",encoding = "utf-8") as fout:
                fout.write(line)
            i = i+1

split_samples(train_token_path,train_samples_path)
split_samples(test_token_path,test_samples_path)

创建数据集Dataset，从文件名称列表中读取文件内容

In [ ]:
class imdbDataset(Dataset):
    def __init__(self,samples_dir):
        self.samples_dir = samples_dir
        self.samples_paths = os.listdir(samples_dir)
    
    def __len__(self):
        return len(self.samples_paths)
    
    def __getitem__(self,index):
        path = self.samples_dir + self.samples_paths[index]
        with open(path,"r",encoding = "utf-8") as f:
            line = f.readline()
            label,tokens = line.split("\t")
            label = torch.tensor([float(label)],dtype = torch.float)
            feature = torch.tensor([int(x) for x in tokens.split(" ")],dtype = torch.long)
            return  (feature,label)
ds_train = imdbDataset(train_samples_path)
ds_test = imdbDataset(test_samples_path)

dl_train = DataLoader(ds_train,batch_size = BATCH_SIZE,shuffle = True,num_workers=4)
dl_test = DataLoader(ds_test,batch_size = BATCH_SIZE,num_workers=4)

for features,labels in dl_train:
    print(features)
    print(labels)
    break

这里数据的样子：
<img style="float: center;" src="images/8.png" width="70%">

features里每一行是一个句子，这里都转成了单词的索引表示，而label里的每一个就是句子对应的分类，好评和差评。

这种方式比较麻烦的就是前面的预处理部分，把数据进行切割，然后构建词典，存到文件等。